In [1]:
import pandas as pd
import numpy as np
import warnings
import pymssql
import locale
import glob
import os

warnings.filterwarnings('ignore')
pd.set_option('use_inf_as_na', True)
pd.options.display.max_columns = None
pd.set_option('float_format', '{:.2f}'.format)
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Код товара], [Название склада], [Название товара], [Артикул поставщика], [Остаток], 
[Проба], [Чистый вес], [Товарное направление], [Товарная группа], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Группа цен], [Скидка на изделии], [Запрет скидки], [Дизайн], [Бренд], [Дата закупки], 
[Цена Розн., за шт], [Гендерный признак], [Ценовая корзина], [Товарный кластер], [Внутренний артикул],
[ID Сайта], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'СИ', N'ИФ', N'ПДК', N'ДК', N'ВЫРИЦА', N'ЧАСЫ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [29]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')

In [3]:
df = stock

In [4]:
ids = df.groupby(by = 'Артикул поставщика', as_index=False)['Остаток'].sum()

In [5]:
ids = ids[~ids['Товарное направление'].str.contains('ФУТЛЯРЫ|БУ')]
ids = ids[~ids['Товарная группа'].str.contains('АКСЕССУАР|ОПТ|ПОСУДА|КРОСС|ЦВ/К|РАЗН|КЕРАМИКА|ЭМАЛЬ|ЯНТАРЬ')]
ids = ids.query('Остаток >= 50')
ids = ids[ids['Проба'].str.contains('585|925')]

In [7]:
idss = df[['Код склада', 'Код товара', 'Название склада', 'Название товара',
    'Артикул поставщика', 'Остаток', 'Проба', 'Чистый вес',
    'Товарное направление', 'Товарная группа', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Группа цен', 'Скидка на изделии',
    'Запрет скидки', 'Дизайн', 'Бренд', 'Дата закупки', 'Цена Розн., за шт',
    'Гендерный признак', 'Ценовая корзина', 'Товарный кластер',
    'Внутренний артикул', 'ID Сайта',
    'Цена Закупки за 1 шт. (упр сс с НДС за единицу)']]

In [8]:
ids = ids.drop_duplicates(subset=['Артикул поставщика'])
ids = idss.merge(ids, how='left', on='Артикул поставщика')

In [10]:
ids = ids.query('Остаток_y >= 50')

In [19]:
articles = ids['Артикул поставщика'].tolist()
arts = ids['ID Сайта'].tolist()

In [30]:
images = images[['article', 'Ссылка на изображение', 'published']]
images = images.drop_duplicates(subset = ['article'])
images['article'] = images['article'].astype(str)
images = images.query('article in @arts')
images.columns = ['ID Сайта', 'Ссылка на изображение', 'published']

In [37]:
ids = ids.merge(images, how='left', on='ID Сайта')

In [38]:
ids.columns

Index(['Код склада', 'Код товара', 'Название склада', 'Название товара',
       'Артикул поставщика', 'Остаток_x', 'Проба', 'Чистый вес',
       'Товарное направление', 'Товарная группа', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Группа цен', 'Скидка на изделии',
       'Запрет скидки', 'Дизайн', 'Бренд', 'Дата закупки', 'Цена Розн., за шт',
       'Гендерный признак', 'Ценовая корзина', 'Товарный кластер',
       'Внутренний артикул', 'ID Сайта',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Остаток_y',
       'Количество', 'Ссылка на изображение', 'published'],
      dtype='object')

In [39]:
ids.columns = ['Код склада', 'Код товара', 'Название склада', 'Название товара',
    'Артикул поставщика', 'Остаток_x', 'Проба', 'Чистый вес',
    'Товарное направление', 'Товарная группа', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Группа цен', 'Скидка на изделии',
    'Запрет скидки', 'Дизайн', 'Бренд', 'Дата закупки', 'Цена Розн., за шт',
    'Гендерный признак', 'Ценовая корзина', 'Товарный кластер',
    'Внутренний артикул', 'ID Сайта',
    'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Остаток_y',
    'Количество продаж за май', 'Ссылка на изображение', 'published']

In [42]:
ids = ids.drop(columns=['Остаток_x'])

In [44]:
ids['price_personal'] = np.where(
    ids['Запрет скидки'] == 'Да', ids['Цена Розн., за шт'], 
    np.where(
        ids['Проба'] == 925, ids['Цена Розн., за шт'] * 0.7 * 0.8 * (1 - (ids['Скидка на изделии'] / 100)), ids['Цена Розн., за шт'] * 0.6 * 0.8 * (1 - (ids['Скидка на изделии'] / 100))
    )
)

In [45]:
ids['Наценка'] = ids['price_personal'] / ids['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] - 1

In [46]:
ids.columns

Index(['Код склада', 'Код товара', 'Название склада', 'Название товара',
       'Артикул поставщика', 'Проба', 'Чистый вес', 'Товарное направление',
       'Товарная группа', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
       'Группа цен', 'Скидка на изделии', 'Запрет скидки', 'Дизайн', 'Бренд',
       'Дата закупки', 'Цена Розн., за шт', 'Гендерный признак',
       'Ценовая корзина', 'Товарный кластер', 'Внутренний артикул', 'ID Сайта',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Остаток_y',
       'Количество продаж за май', 'Ссылка на изображение', 'published',
       'price_personal', 'Наценка'],
      dtype='object')

In [47]:
ids.columns = ['Код склада', 'Код товара', 'Название склада', 'Название товара',
    'Артикул поставщика', 'Проба', 'Чистый вес', 'Товарное направление',
    'Товарная группа', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
    'Группа цен', 'Скидка на изделии', 'Запрет скидки', 'Дизайн', 'Бренд',
    'Дата закупки', 'Цена Розн., за шт', 'Гендерный признак',
    'Ценовая корзина', 'Товарный кластер', 'Внутренний артикул', 'ID Сайта',
    'Себестоимость', 'Остаток',
    'Количество продаж за май', 'Ссылка на изображение', 'published',
    'Итоговая цена на изделии', 'Наценка']

In [48]:
stock['Дата закупки'] = pd.to_datetime(stock['Дата закупки'], dayfirst=True)

In [49]:
min_date = stock.groupby(by = 'Артикул поставщика', as_index=False)['Дата закупки'].min()

In [50]:
min_date.columns = ['article', 'date']

In [51]:
articles = df['Артикул поставщика'].tolist()

In [52]:
min_date = min_date.query('article in @articles')

In [53]:
min_date.columns = ['Артикул поставщика', 'Дата закупки']

In [54]:
ids = ids.drop(columns=['Дата закупки'])

In [55]:
ids = ids.merge(min_date, how='left', on='Артикул поставщика')

In [103]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['2024-05-20_2024-05-31.xlsx']

In [57]:
sale = pd.DataFrame()
for name in all_filenames:
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

In [58]:
sale['Дата'] = pd.to_datetime(sale['Дата'], dayfirst=True, format='mixed')

In [59]:
sale = sale[(sale['Дата'] >= pd.to_datetime('2024-03-14', dayfirst=True, format='mixed')) & 
            (sale['Дата'] <= pd.to_datetime('2024-06-11', dayfirst=True, format='mixed'))]

In [60]:
sale.columns = ['КБК', 'Дата', 'Продавец', 'Номер чека', 'ID продажи', 'Тов',
    'article', 'Количество', 'Номенклатура', 'Номер карты',
    'Товарная группа', 'Товарное направление', 'Тип 1',
    'Списание СберСпасибо', 'Группа наценки', 'Вес', 'Проба', 'Размер',
    'ШК', 'Цена без скидки', 'Базовая скидка', 'Скидка в руб.', 'Название',
    'Цена после скидки', 'Phone', 'Себестоимость', 'Сумма', 'Услуга',
    'Общая сумма']

In [61]:
sale = sale.query('article in @articles')

In [62]:
sale = sale.groupby(by = 'article', as_index=False)[['Количество', 'Общая сумма']].sum()

In [63]:
sale.columns = ['Артикул поставщика', 'Количество продаж', 'Общая сумма продаж']

In [64]:
ids = ids.merge(sale, how='left', on='Артикул поставщика')

In [65]:
ids.columns

Index(['Код склада', 'Код товара', 'Название склада', 'Название товара',
       'Артикул поставщика', 'Проба', 'Чистый вес', 'Товарное направление',
       'Товарная группа', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
       'Группа цен', 'Скидка на изделии', 'Запрет скидки', 'Дизайн', 'Бренд',
       'Цена Розн., за шт', 'Гендерный признак', 'Ценовая корзина',
       'Товарный кластер', 'Внутренний артикул', 'ID Сайта', 'Себестоимость',
       'Остаток', 'Количество продаж за май', 'Ссылка на изображение',
       'published', 'Итоговая цена на изделии', 'Наценка', 'Дата закупки',
       'Количество продаж', 'Общая сумма продаж'],
      dtype='object')

In [66]:
ids.columns = ['Код склада', 'Код товара', 'Название склада', 'Название товара',
    'Артикул поставщика', 'Проба', 'Чистый вес', 'Товарное направление',
    'Товарная группа', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
    'Группа цен', 'Скидка на изделии', 'Запрет скидки', 'Дизайн', 'Бренд',
    'Цена Розн., за шт', 'Гендерный признак', 'Ценовая корзина',
    'Товарный кластер', 'Внутренний артикул', 'ID Сайта', 'Себестоимость',
    'Остаток', 'Количество продаж за май', 'Ссылка на изображение',
    'published', 'Итоговая цена на изделии', 'Наценка', 'Минимальная дата закупки', 'Количество продаж 14.03-11.06', 'Общая сумма продаж 14.03-11.06']

In [69]:
center = stock[stock['Код склада'] == '3075']
center = center.groupby(by = 'Артикул поставщика', as_index=False)['Остаток'].sum()
center.columns = ['Артикул поставщика', 'Остаток на центральном складе']

In [70]:
kolambus = stock[stock['Код склада'] == '5442']
kolambus = kolambus.groupby(by = 'Артикул поставщика', as_index=False)['Остаток'].sum()
kolambus.columns = ['Артикул поставщика', 'Остаток на складе коламбуса']

In [71]:
federal = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Федеральные склады 11.06.xlsx')

In [72]:
federal = federal['outlet_name'].tolist() 

In [73]:
stock.columns = ['Код склада', 'Код товара', 'name', 'Название товара',
    'Артикул поставщика', 'Остаток', 'Проба', 'Чистый вес',
    'Товарное направление', 'Товарная группа', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Группа цен', 'Скидка на изделии',
    'Запрет скидки', 'Дизайн', 'Бренд', 'Дата закупки', 'Цена Розн., за шт',
    'Гендерный признак', 'Ценовая корзина', 'Товарный кластер',
    'Внутренний артикул', 'ID Сайта',
    'Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [74]:
federal = stock.query('name in @federal')

In [75]:
federal = federal.groupby(by = 'Артикул поставщика', as_index=False)['Остаток'].sum()
federal.columns = ['Артикул поставщика', 'Остаток на федеральных складах']

In [76]:
federal

,Артикул поставщика,Остаток на федеральных складах
0,К-392,1.00
1,0-017,5.00
2,00-03-0858,1.00
3,00008-П,1.00
4,00013Ч,1.00
...,...,...
19550,с35-04,6.00
19551,с462-20,2.00
19552,с9-16,3.00
19553,сCRM-026#КЕРАМИК,1.00


In [77]:
ids = ids.merge(federal, how='left', on='Артикул поставщика')

In [78]:
ids = ids.merge(kolambus, how='left', on='Артикул поставщика')

In [79]:
ids = ids.merge(center, how='left', on='Артикул поставщика')

In [80]:
ids.to_excel('предварительный.xlsx', index=False)

In [81]:
ids['Сумма в себестоимости на остатках'] = ids['Себестоимость'] * ids['Остаток']

In [82]:
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')

In [83]:
zapret.columns = ['ТГ', 'article', 'Описание', 'Рейтинг по размещению 30.04.2024',
    'Артикул сайта', 'Опубликован', 'Остаток на 30.04.24',
    'Себестоимость остатка на 30.04.2024',
    'Среднее количество продаж, шт/мес', 'Средняя сумма продаж, руб/мес',
    'Cредняя себестоимость, руб/мес', 'Средняя валовая прибыль,  руб/мес',
    'Средняя цена продажи, шт', 'Причина блокировки']

In [84]:
zapret = zapret['article'].tolist()

In [85]:
ids['Запрет к размещению'] = np.nan
for i in ids.index:
    if ids['Артикул поставщика'][i] in zapret:
        ids['Запрет к размещению'][i] = True
    else:
        ids['Запрет к размещению'][i] = False

In [86]:
ids

,Код склада,Код товара,Название склада,Название товара,Артикул поставщика,Проба,Чистый вес,Товарное направление,Товарная группа,Тип изделия 1,Тип изделия 2,Тип изделия 3,Группа цен,Скидка на изделии,Запрет скидки,Дизайн,Бренд,"Цена Розн., за шт",Гендерный признак,Ценовая корзина,Товарный кластер,Внутренний артикул,ID Сайта,Себестоимость,Остаток,Количество продаж за май,Ссылка на изображение,published,Итоговая цена на изделии,Наценка,Минимальная дата закупки,Количество продаж 14.03-11.06,Общая сумма продаж 14.03-11.06,Остаток на федеральных складах,Остаток на складе коламбуса,Остаток на центральном складе,Сумма в себестоимости на остатках,Запрет к размещению
0,4336,ТОВ1286504,"Вятские Поляны, Мира, 44","Кольцо с брилл., с родир.",20-02-1000-13114,585,1.15,ДК,ДК КОЛЬЦА,КОЛЬЦО,С ДРАГОЦЕННЫМИ КАМНЯМИ,КРАСНЫЙ,ДК_ДЕШЕВАЯ,64.00,Нет,ДК ГЕОМЕТРИЯ,Значение по умолчанию,63234.00,женские,ДЕШЕВАЯ,ДК_2019,1365803 1 1 1_ДК,1689425,5188.22,2326.00,87.00,https://static2.585.cloud/media/products/3a4ad...,True,10926.84,1.11,2023-08-05,277.00,3189708.00,16.00,4.00,23.00,12067799.72,False
1,4336,ТОВ1286507,"Вятские Поляны, Мира, 44","Кольцо с брилл., с родир.+шайба+алм.гр.",20-33-1000-13115,585,1.24,ДК,ДК КОЛЬЦА,КОЛЬЦО,С ДРАГОЦЕННЫМИ КАМНЯМИ,КРАСНЫЙ,ДК_ДЕШЕВАЯ,62.00,Нет,ДК С ОДНИМ КАМНЕМ,Значение по умолчанию,60920.00,женские,ДЕШЕВАЯ,ДК_2016,1398903 1 5 1#БР._ДК,1847403,5152.14,1609.00,610.00,https://static2.585.cloud/media/products/5d363...,True,11111.81,1.16,2023-08-15,1709.00,15465980.00,7.00,31.00,130.00,8289793.26,False
2,4336,ТОВ1286512,"Вятские Поляны, Мира, 44","Кольцо с брилл., с родир.+шайба+алм.гр.",20-33-1000-13117,585,1.22,ДК,ДК КОЛЬЦА,КОЛЬЦО,С ДРАГОЦЕННЫМИ КАМНЯМИ,КРАСНЫЙ,ДК_ДЕШЕВАЯ,67.00,Нет,ДК С ОДНИМ КАМНЕМ,Значение по умолчанию,59575.00,женские,ДЕШЕВАЯ,ДК_2016,1399203 1 5 1#БР._ДК,1865051,4586.64,1778.00,114.00,https://static2.585.cloud/media/products/d9bdb...,True,9436.68,1.06,2023-09-03,318.00,3546368.00,10.00,4.00,17.00,8155045.92,False
3,4336,ТОВ1307981,"Вятские Поляны, Мира, 44",Колье с фиан. род.,TL011624-1,925,3.19,СИ,СИ КОЛЬЕ СК,ЦЕПЬ,С ФИАНИТАМИ,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ_ЦБ_СРЕДН,40.00,Нет,СИ ФАНТАЗИЙКА,Значение по умолчанию,7465.00,женские,ДЕШЕВАЯ,6135_СИ,TL011624-1_СИ,1544525,1480.50,59.00,1.00,NaN,NaN,2149.92,0.45,2023-10-03,2.00,5380.00,1.00,NaN,2.00,87349.50,False
4,4336,ТОВ1308353,"Вятские Поляны, Мира, 44",Цепь Ромб двойной с а/г род.,40-55030-23,925,1.63,СИ,СИ ЦЕПЬ,ЦЕПЬ,БЕЗ КАМНЕЙ,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ_ЦБ_СРЕДН,40.00,Нет,СИ ЦБ РОМБ ДВОЙНОЙ,Значение по умолчанию,1782.00,женские,ДЕШЕВАЯ,6117_СИ,40-55030-23_СИ,1294074,272.20,821.00,45.00,NaN,NaN,513.22,0.89,2023-10-09,107.00,88030.00,3.00,4.00,21.00,223476.20,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17369,3075,ТОВ1307674,Центральный склад Регент Голд,"Кольцо с бриллиантами, с родир.",20-12-1000-13346,585,1.30,ДК,ДК КОЛЬЦА,КОЛЬЦО,С ДРАГОЦЕННЫМИ КАМНЯМИ,БЕЛЫЙ,ДК_ДОРОГАЯ,0.00,Нет,ДК ГЕОМЕТРИЯ,Значение по умолчанию,NaN,женские,ДОРОГАЯ,ДК_2021,20-12-1000-13346_ДК,,0.00,100.00,NaN,NaN,NaN,NaN,NaN,2024-06-18,NaN,NaN,NaN,NaN,100.00,0.00,False
17370,3075,ТОВ1307724,Центральный склад Регент Голд,"Кольцо с бриллиантами+шайба+алм.грань, с родир.",20-33-1000-13351,585,1.61,ДК,ДК КОЛЬЦА,КОЛЬЦО,С ДРАГОЦЕННЫМИ КАМНЯМИ,КРАСНЫЙ,ДК_СРЕДНЯЯ,0.00,Нет,ДК МАЛИНКА,Значение по умолчанию,NaN,женские,СРЕДНЯЯ,ДК_2026,20-33-1000-13351_ДК,,8356.21,100.00,NaN,NaN,NaN,NaN,NaN,2024-06-10,NaN,NaN,NaN,NaN,100.00,835621.00,False
17371,3075,ТОВ1336459,Центральный склад Регент Голд,"Подвес позолота, культивированный жемчуг",18.104,925,2.30,ВЫРИЦА,ВЫРИЦА СЕРЕБРО,ПОДВЕС,С ЖЕМЧУГОМ,ЖЕЛТЫЙ,СИ_ВЫРИЦА_ДЕШ,0.00,Нет,В_ПРОЧЕЕ,ВЫРИЦА,NaN,Значение по умолчанию,СРЕДНЯЯ,,18.104_ВЫРИЦА,,1912.00,101.00,NaN,NaN,NaN,NaN,NaN,2024-06-11,NaN,NaN,NaN,NaN,101.00,193112.00,False
17372,3075,ТОВ1324092,Центральный склад Регент Голд,"Кольцо Спаси и Сохрани с б/ц фиан., род.",20-72-0001-11906,925,2.08,СИ,СИ КОЛЬ

In [88]:
ids['Запрет к размещению'].value_counts()

Запрет к размещению
False    17338
True        36
Name: count, dtype: int64

In [89]:
ids['rank1'] = ids['Остаток на федеральных складах'].rank()
ids['rank2'] = ids['Сумма в себестоимости на остатках'].rank()
ids['rank3'] = ids['Общая сумма продаж 14.03-11.06'].rank()

In [90]:
ids['rank'] = (ids['rank1'] + ids['rank2'] + ids['rank3'])/3

In [91]:
ids = ids.sort_values(by = ['rank'], ascending=False)

In [94]:
# ids = ids.drop(columns=['index'])

In [95]:
ids = ids.reset_index()
ids['Рейтинг'] = np.nan
for i in ids.index:
    ids['Рейтинг'][i] = i + 1

In [96]:
ids = ids.drop(columns=['rank1', 'rank2', 'rank3'])

In [98]:
# df = df.drop(columns=['Скидка на изделии', 'level_0', 'index'])

In [99]:
ids.to_excel('final.xlsx', index=False)

In [ ]:
df = pd.read_excel('final.xlsx')

In [ ]:
ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17363 entries, 0 to 17362
Data columns (total 39 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Код склада                         17363 non-null  object        
 1   Название склада                    17363 non-null  object        
 2   Название товара                    17363 non-null  object        
 3   Артикул поставщика                 17363 non-null  object        
 4   Проба                              17363 non-null  object        
 5   Чистый вес                         17363 non-null  float64       
 6   Товарное направление               17363 non-null  object        
 7   Товарная группа                    17363 non-null  object        
 8   Тип изделия 1                      17363 non-null  object        
 9   Тип изделия 2                      17363 non-null  object        
 10  Тип изделия 3                     